In [1]:
print('Load libraries')

import glob
import os


import itertools

import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.path as mpath
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib.lines import Line2D
import matplotlib.lines as mlines

import statsmodels.api as sm
from scipy import stats

from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
from scipy.stats import linregress

import iris
import iris.quickplot as qplt
import iris.analysis
import iris.coord_categorisation as icoca
from iris.util import mask_cube_from_shapefile
from iris.util import promote_aux_coord_to_dim_coord
from iris.coords import AuxCoord

from typing import List

import cartopy.io.shapereader as shpreader

import pandas as pd
import seaborn as sns

from dataclasses import dataclass

from iris.util import unify_time_units
from iris.util import equalise_attributes

import cmocean

import warnings

import pickle

import datetime

Load libraries


In [2]:
print('Load Custom Functions')

from Cube_Functions import *
from Plot_Functions import *
from Climate_Functions import *
from Stats_Functions import *

Load Custom Functions


In [3]:
import dask.optimization

# Patch the missing class
if not hasattr(dask.optimization, 'SubgraphCallable'):
    class SubgraphCallable:
        def __init__(self, *args, **kwargs):
            pass
    dask.optimization.SubgraphCallable = SubgraphCallable


In [25]:

experiment = 'lig127k'
control = 'piControl'

root_path = '/gws/nopw/j04/pmip4_vol1/public/matt/data/'

months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

t_seconds = 60 * 60 * 24 * 30

t = np.arange(0, 12)

models = ['ACCESS-ESM1-5', 'IPSL-CM6A-LR'] #, 'ACCESS-ESM1-5', 'CESM2', 'IPSL-CM6A-LR', 'NorESM2-LM'] 
models = ['ACCESS-ESM1-5', 'CESM2','EC-Earth3-LR', 'EC-Earth3', 'IPSL-CM6A-LR', 'NESM3', 'NorESM2-LM']

siconc_threshold = 15


In [26]:
con = iris.Constraint(latitude=lambda lat: 60 <= lat <= 90)


In [27]:
time_con = iris.Constraint(year=lambda y:np.logical_and(y>=2035,y<2045))  #time_constraint. Not used anywhere yet.

ocean_shp_reader = shpreader.Reader(shpreader.natural_earth(resolution="110m", category="physical", name="ocean"))
ocean_list = []
for ocean in ocean_shp_reader.records():
    ocean_list.append(ocean.geometry)
ocean_shp = ocean_list[1]

shape = ocean_shp

In [28]:
#Calculate energy from melting
si_density = 917 #kgm-3

#si_specific_volume = si_density * si_specific #kJ per m-3

T_Kelvin = 273.15
T_melt = 0.0    # °C (melting point of sea ice)
T_freeze = -1.8 # °C (freezing point of sea ice)

#salinity = 4.0 #parts per thousand)

#Cox & Weeks (1974) in the Journal of Glaciology ---CHECK THIS!!
#salinity(h) = 7.88 - 1.59h
#average thickness of lig127k, 1.47 => average salinity = 5.38
#average thickness of picontrol, 2.20 => average salinity = 4.38
salinity_dict = {'piControl':2, 'lig127k':5}

#From CICE documents
#https://github.com/CICE-Consortium/CICE-svn-trunk/blob/main/cicedoc/cicedoc.pdf
Co = 2110 #fresh ice specific heat capacity, J per kg
Lo = 334000 #Jkg-1, latent heat of fresh ice
mu = 0.054 #deg ppt-1 ratio between freezing temp and salinity of brine

gamma = Lo * mu #J kg-1 deg-1

In [29]:
warnings.filterwarnings("ignore")

In [30]:
def get_monthly_changes_cube(cube):

    change_cube = cube.copy()
    change_cube.data[1:] = cube.data[1:] - cube.data[:-1]
    change_cube.data[0:0] = change_cube.data[11:11] #don't have the first January, so make it equal to second january. 
    return change_cube



In [31]:
def mid_to_mid_to_month(monthly_data):
    #convert mid to mid monthly data to changes for the month
    monthly_corrected  = (monthly_data + np.roll(monthly_data, 1) ) / 2.0
    return monthly_corrected

In [32]:
def get_specific_energy(experiment):    #get cubes
    cube_dict = {}
    variables = ['tas', 'tos', 'siconc', 'sithick']
    for var in variables:
        path = create_path(model, experiment, var, root_path)
        cube = get_cube(path, var, con, shape)
        cube_dict[var] = cube
    
    
    
    
    tas_cube, tos_cube, siconc_cube, sithick_cube = extract_time_overlap(cube_dict['tas'], 
                                                            cube_dict['tos'],
                                                            cube_dict['siconc'],
                                                            cube_dict['sithick'])
    
    tos_cube = tos_cube.regrid(tas_cube, iris.analysis.Linear())
    si_temp_cube = tas_cube.copy()
    si_temp_cube.data = ((tas_cube.data - T_Kelvin) + tos_cube.data) / 2.0
    
    si_temp_change_cube = get_monthly_changes_cube(si_temp_cube)
    
    #si_temp_change_monthly = get_monthly_data(si_temp_change_cube, 'mean')
    #si_temp_monthly = get_monthly_data(si_temp_cube, 'mean')
    
    #changes in sea ice volume
    #area_weights = iris.analysis.cartography.area_weights(siconc_cube)
    sithick_cube = sithick_cube.regrid(siconc_cube, iris.analysis.Linear())
    si_volume_cube= siconc_cube.copy()
    si_volume_cube.data = ( siconc_cube.data / 100.00 ) * sithick_cube.data  #convert area weights to m2 from km2
    
    end_volume_cube = si_volume_cube.copy()
    end_volume_cube.data[:-1] = si_volume_cube.data[1:]
    
    melt_volume_cube = si_volume_cube.copy()
    melt_volume_cube.data = np.ma.minimum(si_volume_cube.data - end_volume_cube.data, 0.0)
    
    salinity = salinity_dict[experiment]
    
    energy_remaining_cube = si_volume_cube.copy()
    energy_melt_cube = si_volume_cube.copy()
    
    si_temp_change_cube = si_temp_change_cube.regrid(siconc_cube, iris.analysis.Linear())
    si_temp_cube = si_temp_cube.regrid(siconc_cube, iris.analysis.Linear())
    
    temp_for_specific = np.ma.minimum(si_temp_cube.data, T_freeze)
    
    si_specific = Co + (gamma * salinity) / (temp_for_specific * temp_for_specific)
    #si_specific = np.ma.minimum(si_specific, 50000)
    
    si_specific_volume =  si_specific * si_density
    energy_remaining_cube.data = end_volume_cube.data * si_temp_change_cube.data * si_specific_volume / t_seconds
    
    temp_change_for_melt = T_freeze - np.ma.minimum(si_temp_cube.data, T_freeze)
    energy_melt_cube.data = melt_volume_cube.data * temp_change_for_melt * si_specific_volume / t_seconds
    
    #energy_melt_cube.data = melt_volume_cube.data * si_temp_change_cube.data * si_specific_volume / t_seconds
    
    energy_remaining_monthly = get_monthly_data(energy_remaining_cube, 'mean')
    
    
    energy_melt_monthly = get_monthly_data(energy_melt_cube, 'mean')

    energy_remaining_monthly_corrected = mid_to_mid_to_month(energy_remaining_monthly)
    energy_melt_monthly_corrected = mid_to_mid_to_month(energy_melt_monthly)
    
    return energy_remaining_monthly_corrected, energy_melt_monthly_corrected


In [33]:
experiment_specific_dict = {}
control_specific_dict = {}
anomaly_specific_dict = {}

for model in models:
    print(model, end = ' ')
    specific_dict = {}

    print(experiment, end= ' ')
    experiment_remaining_monthly, experiment_melt_monthly = get_specific_energy(experiment)
    experiment_specific_dict[model] = {'remaining':experiment_remaining_monthly,
                                       'melt':experiment_melt_monthly,
                                       'total':experiment_remaining_monthly + experiment_melt_monthly}

    print(control)
    control_remaining_monthly, control_melt_monthly = get_specific_energy(control)
    control_specific_dict[model] = {'remaining':control_remaining_monthly,
                                       'melt':control_melt_monthly,
                                       'total':control_remaining_monthly + control_melt_monthly}
    
    anomaly_remaining_monthly = experiment_remaining_monthly - control_remaining_monthly
    anomaly_melt_monthly = experiment_melt_monthly - control_melt_monthly
    anomaly_specific_dict[model] = {'remaining':anomaly_remaining_monthly,
                                       'melt':anomaly_melt_monthly,
                                       'total':anomaly_remaining_monthly + anomaly_melt_monthly}

    
    

ACCESS-ESM1-5 lig127k piControl
CESM2 lig127k 

ValueError: operands could not be broadcast together with shapes (8399,33,288) (8400,33,288) 

In [ ]:
#experiment_remaining_monthly 
#experiment_melt_monthly
#control_remaining_monthly
#control_melt_monthly

#anomaly_total_monthly

salinity_string = f"_{salinity_dict['piControl']}_{salinity_dict['lig127k']}"


In [ ]:
total_anomaly_mmm = plot_multi_model_mean(anomaly_specific_dict, 'total', 
                                          title = 'Specific Heat of Sea Ice Anomaly', 
                                          ylabel='Anomaly (Wm⁻²)', 
                                          save_name='si_specific_mmm'+salinity_string)
total_anomaly_mmm

#salinity 2 & 5
#array([-0.65776993, -0.54464492, -0.78253749, -0.69690401,  7.20677304,
#        8.11397321, -1.78945628, -1.82488379,  2.14191912,  2.06142482,
#        0.549994  , -0.44004284])

In [ ]:

#sea ice present at end of month
# energy =  volume_still_there * change_in_T_over_the_month * specific_heat_volume

#sea ice that's melted
# energy = volume_melted * (melt_T - np.minimum(tas_at_start, 0)) * specific_heat_volume    ###assume that melt occurs at the top and so is ~at T

#sea_ice_that's frozen
# assume no energy change - SST kept close to freezing point

# From Bitz and Lipscomb (1999)
# Heat to warm a cubic metre of sea ice is given by  - 
# Q = rho * [Co *(T' - T) - Lo*mu*S*(1/T' - 1/T)]
#where 
#rho = density
#Co = Specific heat capacity, 2110 J kg-1 deg-1
#Lo = Latent heat of fusion of fresh ice
#mu = 0.054 degrees C
#S = salintiy in parts per thousand

#alternatively, can express this as an alteration in the specific heat, Co such that
#C[sea_ice] = Co + gamma * S / T^2
#where gamma = 17200 J deg-1 kg-1

#For high salinity, warm sea ice this will have a meaningful impact. 
#LIG sea ice will be warmer AND potentially more saline since less time for brine rejection. 
#THIS COULD BE VERY IMPORTANT!!!!

#How can I get the salinity of the sea ice?!

Investigate this further:

In climate models or reanalyses, sea ice temperature is often approximated using surface energy balance or empirical relationships. A common first-order estimate is:

T_ice ≈ α * T_air + (1 - α) * T_ocean
Where:

α ≈ 0.7 to 0.9 in winter (more weight on air temp)

α ≈ 0.5 to 0.6 in shoulder seasons

In summer, T_ice approaches 0°C regardless of SAT or SST

This is where your approximation — taking the average of TAS and TOS — is a reasonable first-order assumption, especially if you have no vertical information on ice temperature.

📘 See also:

Maykut and Untersteiner (1971): standard model of sea ice thermodynamics

Bitz and Lipscomb (1999): multi-layer model in CICE

Semtner (1976): zero-layer model for GCMs